# Supervised Learning of Actions - Logistic Regression
This note fits a logistic regression model to the sepsis data.

In [1]:
import pandas as pd
import numpy as np
exportdir='/data/localhost/taufiq/export-dir/'

In [2]:
import logging
logging.basicConfig(filename='logistic-regression.log', level=logging.INFO)

In [3]:
nra = 5
nr_reps = 50

In [4]:
MIMICtable = pd.read_csv(exportdir + '/MIMICtable.csv')
MIMICtable.head()

,bloc,icustay_id,charttime,gender,age,elixhauser,re_admission,died_in_hosp,died_within_48h_of_out_time,mortality_90d,...,mechvent,Shock_Index,PaO2_FiO2,median_dose_vaso,max_dose_vaso,input_total,input_4hourly,output_total,output_4hourly,cumulated_balance
0,1.0,3.0,7.245400e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,...,0.0,0.653782,222.499997,0.0,0.0,6297.0,30.0,9090.0,4305.0,-2793.0
1,2.0,3.0,7.245414e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,...,0.0,0.710438,207.499997,0.0,0.0,6347.0,50.0,13060.0,3970.0,-6713.0
2,3.0,3.0,7.245428e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,...,0.0,0.748397,207.499997,0.0,0.0,6397.0,50.0,16260.0,3200.0,-9863.0
3,4.0,3.0,7.245443e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,...,0.0,0.615226,207.499997,0.0,0.0,6447.0,50.0,18910.0,2650.0,-12463.0
4,5.0,3.0,7.245457e+09,0.0,17639.826435,0.0,0.0,0.0,0.0,1.0,...,0.0,0.749047,165.573772,0.0,0.0,6477.0,30.0,21180.0,2270.0,-14703.0


In [5]:
#################   Convert training data and compute conversion factors    ######################
logging.info('Processing raw data')
# all 47 columns of interest
colbin = ['gender','mechvent','re_admission']
colnorm=['age','Weight_kg','GCS','HR','SysBP','MeanBP','DiaBP','RR','Temp_C','FiO2_1',\
    'Potassium','Sodium','Chloride','Glucose','Magnesium','Calcium',\
    'Hb','WBC_count','Platelets_count','PTT','PT','Arterial_pH','paO2','paCO2',\
    'Arterial_BE','HCO3','Arterial_lactate','SOFA','SIRS','Shock_Index','PaO2_FiO2','cumulated_balance']
collog=['SpO2','BUN','Creatinine','SGOT','SGPT','Total_bili','INR','output_total','output_4hourly']

MIMICraw = MIMICtable[colbin+colnorm+collog].copy()

for col in MIMICraw:
    if col in colbin:
        MIMICraw[col] = MIMICraw[col] - 0.5
    elif col in colnorm:
        cmu = MIMICraw[col].mean()
        csigma = MIMICraw[col].std()
        MIMICraw[col] = (MIMICraw[col] - cmu)/csigma
    else:
        log_values = np.log(0.1 + MIMICraw[col])
        dmu = log_values.mean()
        dsigma = log_values.std()
        MIMICraw[col] = (log_values - dmu)/dsigma    
logging.info('Raw data processed')

## Include the information for doses taken until time $t$

In [6]:
logging.info('Adding input values to X')

MIMICraw['last_input'] = 0.0
MIMICraw['total_input_before'] = 0.0
MIMICraw['last_vaso_dose'] = 0.0

for index, row in MIMICtable.iterrows():
    if index < len(MIMICtable) - 1 and (MIMICtable.at[index+1,'icustay_id'] == MIMICtable.at[index,'icustay_id']):
        MIMICraw.at[index+1, 'last_input'] = MIMICtable.at[index, 'input_4hourly']
        MIMICraw.at[index+1, 'total_input_before'] = MIMICtable.at[index, 'input_total']
        MIMICraw.at[index+1, 'last_vaso_dose'] = MIMICtable.at[index, 'max_dose_vaso']

def log_normalise(colname):
    global MIMICraw
    log_values = np.log(0.1 + MIMICraw[colname])
    dmu = log_values.mean()
    dsigma = log_values.std()
    MIMICraw[colname] = (log_values - dmu)/dsigma

log_normalise('last_input')
log_normalise('total_input_before')
MIMICraw['last_vaso_dose'] = MIMICraw['last_vaso_dose'] - 0.5

logging.info('Input values added to X')

In [7]:
MIMICraw.to_csv(exportdir + '/MIMICraw-logistic_reg.csv', index = False)
MIMICraw.head()

,gender,mechvent,re_admission,age,Weight_kg,GCS,HR,SysBP,MeanBP,DiaBP,...,Creatinine,SGOT,SGPT,Total_bili,INR,output_total,output_4hourly,last_input,total_input_before,last_vaso_dose
0,-0.5,-0.5,-0.5,-1.051213,1.827456,-0.952846,-0.523975,-0.064338,0.296909,0.475162,...,0.735987,0.250377,-0.38494,3.417208,0.302317,0.074017,0.612008,-1.227250,-1.967244,-0.5
1,-0.5,-0.5,-0.5,-1.051213,-0.135125,-0.952846,-0.445075,-0.441478,0.000394,0.195434,...,-0.730676,-0.171064,-0.38494,1.549167,-0.961687,0.182853,0.590004,0.287835,0.627278,-0.5
2,-0.5,-0.5,-0.5,-1.051213,-0.135125,-0.899877,-0.521990,-0.806252,-0.147864,0.182833,...,-0.730676,-0.171064,-0.38494,1.549167,-0.245226,0.248674,0.531440,0.423093,0.629135,-0.5
3,-0.5,-0.5,-0.5,-1.051213,-0.135125,-0.635036,-0.705596,0.059314,1.334712,1.745278,...,-0.730676,-0.171064,-0.38494,1.549167,-0.961687,0.294020,0.480217,0.423093,0.630978,-0.5
4,-0.5,-0.5,-0.5,-1.051213,-0.135125,0.381956,0.690797,0.534139,1.527447,1.767959,...,-0.730676,-0.171064,-0.38494,1.549167,-0.961687,0.328069,0.438177,0.423093,0.632806,-0.5


## Create Actions


In [8]:
from sklearn.cluster import KMeans
from scipy.stats import rankdata

logging.info('Creating action bins')
nact = nra**2
input_4hourly_nonzero = MIMICtable.loc[MIMICtable['input_4hourly']>0, 'input_4hourly']
iol_ranked = rankdata(input_4hourly_nonzero)/len(input_4hourly_nonzero) # excludes zero fluid (will be action 1)
iof = np.floor((iol_ranked + 0.2499999999)*4) # converts iv volume in 4 actions
io = np.ones(len(MIMICtable)) # array of ones, by default
io[MIMICtable['input_4hourly']>0] = iof + 1 # where more than zero fluid given: save actual action
vc = MIMICtable['max_dose_vaso'].copy()
vc_nonzero = MIMICtable.loc[MIMICtable['max_dose_vaso']!=0, 'max_dose_vaso']
vc_ranked = rankdata(vc_nonzero)/len(vc_nonzero)
vcf = np.floor((vc_ranked + 0.2499999999)*4) # converts to 4 bins
vcf[vcf==0] = 1
vc[vc!=0] = vcf + 1
vc[vc==0] = 1
# median dose of drug in all bins
ma1 = [MIMICtable.loc[io==1, 'input_4hourly'].median(), MIMICtable.loc[io==2, 'input_4hourly'].median(), MIMICtable.loc[io==3, 'input_4hourly'].median(), MIMICtable.loc[io==4, 'input_4hourly'].median(), MIMICtable.loc[io==5, 'input_4hourly'].median()]
ma2 = [MIMICtable.loc[vc==1, 'max_dose_vaso'].median(), MIMICtable.loc[vc==2, 'max_dose_vaso'].median(), MIMICtable.loc[vc==3, 'max_dose_vaso'].median(), MIMICtable.loc[vc==4, 'max_dose_vaso'].median(), MIMICtable.loc[vc==5, 'max_dose_vaso'].median()]
med = pd.DataFrame(data={'IV':io, 'VC': vc})
med = med.astype({'IV': 'int32', 'VC': 'int32'})
uniqueValues = med.drop_duplicates().reset_index(drop=True)
uniqueValueDoses = pd.DataFrame()
for index, row in uniqueValues.iterrows():
    uniqueValueDoses.at[index, 'IV'], uniqueValueDoses.at[index, 'VC'] = ma1[row['IV']-1], ma2[row['VC']-1]

actionbloc = pd.DataFrame()
for index, row in med.iterrows():
    actionbloc.at[index, 'action_bloc'] = uniqueValues.loc[(uniqueValues['IV'] == row['IV']) & (uniqueValues['VC'] == row['VC'])].index.values[0]+1
actionbloc = actionbloc.astype({'action_bloc':'int32'})

logging.info('Action bins created')

## Fitting models

In [ ]:
from sklearn.linear_model import LogisticRegression
icuuniqueids = MIMICtable['icustay_id'].unique()
modelsDf = pd.DataFrame()

logging.info('Fitting models')

for model in range(nr_reps):
    logging.info('Model: ' + str(model))
    grp = np.floor(5*np.random.rand(len(icuuniqueids))+1)
    crossval = 1
    trainidx = icuuniqueids[grp != crossval]
    testidx = icuuniqueids[grp == crossval]
    X = MIMICraw.loc[MIMICtable['icustay_id'].isin(trainidx)]
    Xtestmimic = MIMICraw[MIMICtable['icustay_id'].isin(testidx)]
    blocs = MIMICtable.loc[MIMICtable['icustay_id'].isin(trainidx), 'bloc']
    bloctestmimic = MIMICtable.loc[MIMICtable['icustay_id'].isin(testidx), 'bloc']
    ptid = MIMICtable.loc[MIMICtable['icustay_id'].isin(trainidx), 'icustay_id']
    ptidtestmimic = MIMICtable.loc[MIMICtable['icustay_id'].isin(testidx), 'icustay_id']
    Y = actionbloc.loc[MIMICtable['icustay_id'].isin(trainidx), 'action_bloc']
    Ytest = actionbloc.loc[MIMICtable['icustay_id'].isin(testidx), 'action_bloc']
    clf = LogisticRegression(random_state=0, max_iter=100000).fit(X, Y)
    acc_train = clf.score(X, Y)
    acc_test = clf.score(Xtestmimic, Ytest)
    modelsDf = modelsDf.append({'model': model, 'regressor': clf, 'acc_train': acc_train, 'acc_test': acc_test}, ignore_index=True)
    
logging.info('Model fitting done!')

In [12]:
modelsDf

,acc_test,acc_train,model,regressor
0,0.492631,0.499326,0.0,"LogisticRegression(C=1.0, class_weight=None, d..."
1,0.497035,0.498887,1.0,"LogisticRegression(C=1.0, class_weight=None, d..."
2,0.486220,0.500617,2.0,"LogisticRegression(C=1.0, class_weight=None, d..."
3,0.490288,0.499413,3.0,"LogisticRegression(C=1.0, class_weight=None, d..."
4,0.490287,0.499687,4.0,"LogisticRegression(C=1.0, class_weight=None, d..."
5,0.481178,0.501231,5.0,"LogisticRegression(C=1.0, class_weight=None, d..."
6,0.489664,0.499725,6.0,"LogisticRegression(C=1.0, class_weight=None, d..."
7,0.486765,0.501278,7.0,"LogisticRegression(C=1.0, class_weight=None, d..."
8,0.478157,0.501337,8.0,"LogisticRegression(C=1.0, class_weight=None, d..."
9,0.493081,0.499578,9.0,"LogisticRegression(C=1.0, class_weight=None, d..."
